# Processo de consolidação e escrita de modalidades 
Choose one of the abbreviations in modality variable

Abbreviation	Category
TT	Total
CRT	Cartão de Crédito
VEIC	Financiamento de veículo
CP	Crédito Pessoal
CHE	Cheque Especial
CNG	Empréstimo Consignado
MCC	Microcrédito
IMB	Crédito Imobiliário
OLM	Outros Limites
DMS Demais Créditos


In [ ]:
from pyspark import SparkConf
from pyspark.sql import SparkSession
import botocore.session

import pyspark
from pyspark.sql.functions import *
from pyspark.sql.types import StructType,StructField, StringType, IntegerType 
from pyspark.sql.types import ArrayType, DoubleType, BooleanType
from pyspark.sql.functions import col,array_contains
from pyspark.sql.functions import sum
from pyspark.sql import functions as F
from operator import add, and_, sub
from functools import reduce

In [ ]:
bucket = "modelagem"
data_key = "scr/processed-data"
data_location = f"s3://{bucket}/{data_key}"

#s3://modelagem/scr/processed-data/CHE/

DEFINIÇÕA DO MÊS DE LEITURA DAS BASES DE DADOS PARA CARGA INCREMENTAL

In [ ]:
dtbase = '202111'

LEITURA DOS DATABASES POR CATEGORIA 

In [ ]:
df_scr_che = spark.read.parquet(f'{data_location}/CHE')

In [ ]:
df_scr_cng = spark.read.parquet(f'{data_location}/CNG')

In [ ]:
df_scr_cp = spark.read.parquet(f'{data_location}/CP')

In [ ]:
df_scr_crt = spark.read.parquet(f'{data_location}/CRT')

In [ ]:
df_scr_imb = spark.read.parquet(f'{data_location}/IMB')

In [ ]:
df_scr_dms = spark.read.parquet(f'{data_location}/DMS')

In [ ]:
df_scr_mcc = spark.read.parquet(f'{data_location}/MCC')

In [ ]:
df_scr_olm = spark.read.parquet(f'{data_location}/OLM')

In [ ]:
df_scr_tt = spark.read.parquet(f'{data_location}/TT')

In [ ]:
df_scr_veic = spark.read.parquet(f'{data_location}/VEIC')

CONSOLIDAÇÃO DAS CATEGORIAS VIA LEFT JOIN DAS BASES TOTAIS COM AS DEMAIS CATEGORIAS

In [ ]:
df_scr_che = df_scr_che.filter(df_scr_che.DTBASE == dtbase)

In [ ]:
df_scr_cng = df_scr_cng.filter(df_scr_cng.DTBASE == dtbase)

In [ ]:
df_scr_cp = df_scr_cp.filter(df_scr_cp.DTBASE == dtbase)

In [ ]:
df_scr_crt = df_scr_crt.filter(df_scr_crt.DTBASE == dtbase)

In [ ]:
df_scr_imb = df_scr_imb.filter(df_scr_imb.DTBASE == dtbase)

In [ ]:
df_scr_dms = df_scr_dms.filter(df_scr_dms.DTBASE ==  dtbase)

In [ ]:
df_scr_mcc = df_scr_mcc.filter(df_scr_mcc.DTBASE ==  dtbase)

In [ ]:
df_scr_olm = df_scr_olm.filter(df_scr_olm.DTBASE == dtbase)

In [ ]:
df_scr_tt = df_scr_tt.filter(df_scr_tt.DTBASE == dtbase)

In [ ]:
df_scr_veic = df_scr_veic.filter(df_scr_veic.DTBASE == dtbase)

In [ ]:
df_consolidado = df_scr_tt.join(df_scr_crt,["DTBASE","CD"],"left")

In [ ]:
df_consolidado = df_consolidado.join(df_scr_che,["DTBASE","CD"],"left")

In [ ]:
df_consolidado = df_consolidado.join(df_scr_cng,["DTBASE","CD"],"left")

In [ ]:
df_consolidado = df_consolidado.join(df_scr_cp,["DTBASE","CD"],"left")

In [ ]:
df_consolidado = df_consolidado.join(df_scr_imb,["DTBASE","CD"],"left")

In [ ]:
df_consolidado = df_consolidado.join(df_scr_mcc,["DTBASE","CD"],"left")

In [ ]:
df_consolidado = df_consolidado.join(df_scr_olm,["DTBASE","CD"],"left")

In [ ]:
df_consolidado = df_consolidado.join(df_scr_veic,["DTBASE","CD"],"left")

In [ ]:
df_consolidado = df_consolidado.join(df_scr_dms,["DTBASE","CD"],"left")

ELIMINAÇÃO DAS COLUNAS DE MXXX DEVIDO A DUPLICIDADE DAS MESMAS E DA NÃO NECESSIDADE PARA ANÁLISES

In [ ]:
df_consolidado = df_consolidado.drop("M0","M1","M2","M3","M4","M5")

ESCRITA DA BASE DE DADOS CONSOLIDADA EM PARQUET

In [ ]:
df_consolidado.write.format("parquet").mode("append").save("s3://modelagem/scr/processed-data/consolidado/")